## Import Library

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Load Price Data

In [3]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    raw_data = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    raw_data = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)
raw_data.tail(10)
ticker_data = raw_data[["Open", "Close", "Low", "High", "Volume"]]
ticker_data.columns = ["open", "close", "low", "high", "volume"]

In [4]:
#@title Raw Price
def cal_first_close(tick):
  if not tick.empty:
    return tick[0]


def cal_high(tick):
  tick = tick[100*tick.index.hour+tick.index.minute > 925]
  tick = tick[100*tick.index.hour+tick.index.minute < 1355]
  return tick.max()


def cal_low(tick):
  tick = tick[100*tick.index.hour+tick.index.minute > 925]
  tick = tick[100*tick.index.hour+tick.index.minute < 1355]
  return tick.min()


def cal_price(tick):
  tick = tick[100*tick.index.hour+tick.index.minute == 1355]
  if not tick.empty:
    return tick[0]

def cal_close(tick):
  tick = tick[100*tick.index.hour+tick.index.minute == 1425]
  if not tick.empty:
    return tick[0]

price = ticker_data
price['first_close'] = price.close
price['price'] = price.close
price = price.resample("D").agg({
    'first_close': cal_first_close,
    'high':cal_high,
    'low': cal_low,
    'price': cal_price,
    'close': cal_close
    })
raw = price.dropna()

In [5]:
price = raw.copy()
price['percent'] = 100 * (price.price - price.close.shift(1)) / price.close.shift(1)
price['returns'] = (price.price - price.first_close) / (price.high - price.low)
price['return'] = 1000 * (price.close - price.price) / price.price

In [6]:
price['percent_group'] = pd.qcut(price.percent, 30)
price['returns_group'] = pd.qcut(price.returns, 30)
price['momentum'] = price.percent + price.returns

## Strategy

In [7]:
def trade_simulation(prepared_data):
    _trades = pd.DataFrame(columns=("EntryTime", "EntryPrice", "ExitTime", "ExitPrice", "Type", "Profit"))
    has_open_deal = False
    type_open_deal = ''
    # Stoploss at 20 pips
    sl_step = 3
    # Takeprofit at 60 pips(R/R = 1/3)
    tp_step = 6
    for i, row in prepared_data.iterrows():
        if 100*row.name.hour + row.name.minute == 1355:
            if row['signal'] == 'short':
                # Open short deal
                entry_price = row['Close']
                entry_time = row.name
                has_open_deal = True
                type_open_deal = 'short'
            elif row['signal'] == 'long':
                # Open short deal
                entry_price = row['Close']
                entry_time = row.name
                has_open_deal = True
                type_open_deal = 'long'
        else:
            if 1400 < 100*row.name.hour + row.name.minute < 1425:
                if has_open_deal:
                    if type_open_deal == 'short':
                        # Stoploss
                        if row['High'] > entry_price + sl_step:
                            profit = -sl_step
                            exit_price = entry_price + sl_step
                            exit_time = row.name
                            _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                            has_open_deal = False
                            type_open_deal = ''
                        # else:
                        #     # Takeprofit
                        #     if row['Low'] < entry_price - tp_step:
                        #         profit = 6
                        #         exit_price = entry_price - tp_step
                        #         exit_time = row.name
                        #         _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                        #         has_open_deal = False
                        #         type_open_deal = ''
                    elif type_open_deal == 'long':
                        # Stoploss
                        if row['Low'] < entry_price - sl_step:
                            profit = -sl_step
                            exit_price = entry_price - sl_step
                            exit_time = row.name
                            _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
                            has_open_deal = False
                            type_open_deal = ''
                        # else:
                        #     # Takeprofit
                        #     if row['High'] > entry_price + tp_step:
                        #         profit = 6
                        #         exit_price = entry_price + tp_step
                        #         exit_time = row.name
                        #         _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
                        #         has_open_deal = False
                        #         type_open_deal = ''
                            #
            elif 100*row.name.hour + row.name.minute == 1425:
                if has_open_deal:
                    # close open deal at 2:25PM (dataframe)
                    if type_open_deal == 'short':
                        profit = entry_price - row['Close']
                        if profit < -sl_step:
                            profit = -sl_step
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                    elif type_open_deal == 'long':
                        profit = row['Close'] - entry_price
                        if profit < -sl_step:
                            profit = -sl_step
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
    return _trades

In [8]:
data3 = price.copy()
def cal_signal_01(row):
    signal = ''
    if row['percent'] > .26 and row['returns'] > .39:
        signal = 'long'
    elif row['percent'] < -.18 and row['returns'] < -.39:
        signal = 'short'
    return signal
data3['signal'] = data3.apply(lambda r: cal_signal_01(r), axis=1)
data4 = raw_data.copy()
data4 = data4.assign(time_d=pd.PeriodIndex(data4.index, freq='1D').to_timestamp())
data4 = pd.merge(data4, data3, left_on="time_d", right_index=True, how="left")

In [9]:
trade02_result = trade_simulation(data4)

In [10]:
trade02_result.Profit.sum()

1165.4999999999986

In [11]:
trade02_result

,EntryTime,EntryPrice,ExitTime,ExitPrice,Type,Profit
0,2018-08-14 13:55:00,959.5,2018-08-14 14:25:00,960.1,Long,0.6
1,2018-08-21 13:55:00,950.1,2018-08-21 14:25:00,954.7,Long,4.6
2,2018-08-23 13:55:00,962.6,2018-08-23 14:25:00,961.0,Long,-1.6
3,2018-08-27 13:55:00,963.7,2018-08-27 14:25:00,963.0,Long,-0.7
4,2018-08-29 13:55:00,962.3,2018-08-29 14:25:00,963.9,Short,-1.6
...,...,...,...,...,...,...
705,2024-06-21 13:55:00,1316.8,2024-06-21 14:10:00,1319.8,Short,-3.0
706,2024-06-24 13:55:00,1296.8,2024-06-24 14:25:00,1287.1,Short,9.7
707,2024-06-28 13:55:00,1283.9,2024-06-28 14:10:00,1286.9,Short,-3.0
708,2024-07-02 13:55:00,1297.1,2024-07-02 14:10:00,1294.1,Long,-3.0


In [12]:
trading_fee = len(trade02_result) * 0.3
trading_fee

213.0

In [13]:
# win rate
len(trade02_result[trade02_result.Profit > 0]) / (len(trade02_result[trade02_result.Profit < 0]) + len(trade02_result[trade02_result.Profit > 0]))

0.5014245014245015

In [14]:
this_month = trade02_result[(trade02_result.EntryTime > '2024-06-01 00:00:00') & (trade02_result.EntryTime < '2024-06-30 15:00:00')]
this_month.Profit.sum()

12.100000000000136

In [15]:
this_year = trade02_result[(trade02_result.EntryTime > '2024-01-01 00:00:00') & (trade02_result.EntryTime < '2024-06-30 15:00:00')]
this_year.Profit.sum()

87.70000000000073

In [16]:
trade02_result.Profit.max()

39.09999999999991

In [17]:
trade02_result.Profit.min()

-3.0

In [18]:
trade02_result.Profit.mean()

1.641549295774646

## Long

In [19]:
Long = price[price.percent > .26]
Long = Long[Long.returns > .39]
Long

,first_close,high,low,price,close,percent,returns,return,percent_group,returns_group,momentum
Date,,,,,,,,,,,
2018-08-14,954.9,958.6,952.1,959.5,960.1,0.565978,0.707692,0.625326,"(0.472, 0.585]","(0.707, 0.782]",1.273671
2018-08-21,944.5,950.8,944.5,950.1,954.7,0.571610,0.888889,4.841596,"(0.472, 0.585]","(0.871, 0.933]",1.460499
2018-08-23,957.0,959.9,955.8,962.6,961.0,0.785258,1.365854,-1.662165,"(0.761, 0.907]","(1.068, 1.636]",2.151112
2018-08-27,961.5,965.2,961.5,963.7,963.0,0.500574,0.594595,-0.726367,"(0.472, 0.585]","(0.556, 0.644]",1.095168
2018-09-07,934.6,942.6,930.5,941.3,944.8,0.630746,0.553719,3.718262,"(0.585, 0.668]","(0.476, 0.556]",1.184465
...,...,...,...,...,...,...,...,...,...,...,...
2024-06-03,1281.4,1298.4,1281.4,1296.6,1293.7,2.231333,0.894118,-2.236619,"(2.219, 7.145]","(0.871, 0.933]",3.125451
2024-06-12,1303.1,1312.9,1302.5,1313.9,1328.4,0.435713,1.038462,11.035847,"(0.393, 0.472]","(0.933, 1.068]",1.474175
2024-06-18,1311.9,1319.7,1311.4,1321.4,1309.5,0.877930,1.144578,-9.005600,"(0.761, 0.907]","(1.068, 1.636]",2.022508


In [20]:
Long['return'].sum()

459.33014500646885

## Short

In [21]:
Short = price[price.percent < -.18]
Short = Short[Short.returns < -.39]

In [22]:
Short

,first_close,high,low,price,close,percent,returns,return,percent_group,returns_group,momentum
Date,,,,,,,,,,,
2018-08-29,965.7,965.9,963.2,962.3,963.9,-0.372709,-1.259259,1.662683,"(-0.469, -0.373]","(-2.006, -1.044]",-1.631969
2018-09-05,952.7,954.4,945.6,947.4,942.5,-1.003135,-0.602273,-5.172050,"(-1.096, -0.879]","(-0.665, -0.595]",-1.605408
2018-09-06,945.0,945.8,936.6,935.6,935.4,-0.732095,-1.021739,-0.213767,"(-0.879, -0.717]","(-1.044, -0.914]",-1.753835
2018-09-10,943.5,944.8,936.3,936.1,938.0,-0.920830,-0.870588,2.029698,"(-1.096, -0.879]","(-0.914, -0.798]",-1.791418
2018-10-05,990.4,990.8,985.3,987.1,982.7,-0.182020,-0.600000,-4.457502,"(-0.209, -0.149]","(-0.665, -0.595]",-0.782020
...,...,...,...,...,...,...,...,...,...,...,...
2024-06-14,1332.1,1335.9,1329.8,1329.7,1326.6,-0.575744,-0.393443,-2.331353,"(-0.717, -0.572]","(-0.406, -0.34]",-0.969187
2024-06-17,1313.9,1314.4,1306.4,1309.7,1309.9,-1.273933,-0.525000,0.152707,"(-1.477, -1.096]","(-0.595, -0.521]",-1.798933
2024-06-21,1321.2,1321.7,1314.5,1316.8,1317.5,-0.513750,-0.611111,0.531592,"(-0.572, -0.469]","(-0.665, -0.595]",-1.124861


In [23]:
abs(Short['return'].sum())

596.8668025859007